In [8]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from pydantic import create_model
import inspect,json
from inspect import Parameter
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import (
    convert_to_openai_function,
    convert_to_openai_tool,
)
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

import random
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
print(load_dotenv())
import os
import openai

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

True


## Building Custom Agent

Defining the function

In [21]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "Compute a^2 - b^2 that is 'difference of square of numbers' between two numbers"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = True

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a**2 - b**2

In [22]:
search = CustomCalculatorTool()
print(search.name)
print(search.description)
print(search.args)

Calculator
Compute a^2 - b^2 that is 'difference of square of numbers' between two numbers
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


In [38]:
@tool(return_direct=True)
def squared_num_diff(num1:int, num2:int) -> int:
    "Compute a^2 - b^2 that is 'difference of square of numbers' between two numbers"
    return num1**2 - num2**2
    

In [39]:
search.run({'a':3,'b':2})

5

In [40]:
squared_num_diff

StructuredTool(name='squared_num_diff', description="Compute a^2 - b^2 that is 'difference of square of numbers' between two numbers", args_schema=<class 'pydantic.v1.main.squared_num_diffSchema'>, return_direct=True, func=<function squared_num_diff at 0x0000026932F74CA0>)

In [49]:
squared_num_diff({int1:2,int2=1})

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (613303032.py, line 1)

In [25]:
json_squared = convert_to_openai_function(CustomCalculatorTool)

TypeError: Object of type 'ModelMetaclass' is not JSON serializable

In [14]:
MODEL = 'llama3-groq-70b-8192-tool-use-preview'
user_prompt = "what is 4^square -1^square?"

messages=[
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        # stream=False
        tools=json_squared, # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_tokens=4096 # Maximum number of tokens to allow in our response
    )
print(response)

BadRequestError: Error code: 400 - {'error': {'message': "'tools.0' : value must be an object", 'type': 'invalid_request_error'}}

In [10]:
# from groq import Groq
# import json

# # Initialize the Groq client
# client = Groq()
# response = client.chat.completions.create(
#         model=MODEL, # LLM to use
#         messages=messages, # Conversation history
#         # stream=False
#         # tools=tools, # Available tools (i.e. functions) for our LLM to use
#         tool_choice="auto", # Let our LLM decide when to use tools
#         max_tokens=4096 # Maximum number of tokens to allow in our response
#     )
# print(response)